# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [1]:
!pwd

/home/jovyan/work/Github/predicting_stock_market_trends_with_Twitter/ipynb


In [2]:
from os import chdir
chdir('/home/jovyan/work/Github/predicting_stock_market_trends_with_Twitter/')

from lib import *
suppress_warnings()

### Create database with PyMongo and install Tweepy:

In [3]:
!pip install pymongo tweepy

    100% |████████████████████████████████| 368kB 173kB/s ta 0:00:01    94% |██████████████████████████████  | 337kB 5.4MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 3.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for oauthlib ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/84/98/7a/fba7268f61097bea6081cbe5480bc439b38975748ea7684fd5
Successfully built oauthlib
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pymongo
cli = pymongo.MongoClient(host='35.163.253.8')

In [5]:
cli.drop_database('twitter_db')

In [6]:
# Create a reference. This is not going to instantiate until you put data in it:
twitter_db = cli.twitter_db
cli.database_names()

['admin', 'local']

In [7]:
twitter_collection = cli.twitter_db.twitter_collection
cli.database_names()

['admin', 'local']

### Pull data from Twitter API:

In [8]:
import tweepy

In [9]:
from lib.twitter_keys import my_keys

In [10]:
#Variables that contains the user credentials to access Twitter API 
access_token = my_keys['ACCESS_TOKEN']
access_token_secret = my_keys['ACCESS_SECRET']
consumer_key = my_keys['CONSUMER_KEY']
consumer_secret = my_keys['CONSUMER_SECRET']

In [11]:
twitter_30_df = pd.read_csv('/home/jovyan/work/Github/predicting_stock_market_trends_with_Twitter/data/twitter_users_30.csv')
twitter_30_df.sample(3)

,Name,Handle,Followers,Description,Type,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
23,Carl Icahn,Carl Icahn,"332,000","Chairman of Icahn Enterprises L.P.; etc., etc....",Thought Leader,NaN,NaN,NaN,NaN
5,Ashton Kutcher,aplusk,"18,100,000",Celebrity,Celebrity,NaN,NaN,NaN,NaN
10,Mark Cuban,mcuban,"7,040,000",Tech investor,Celebrity,NaN,NaN,NaN,NaN


In [12]:
handle = list(twitter_30_df['Handle'])

In [13]:
handle

['BarackObama',
 'cnnbrk',
 'BillGates',
 'realDonaldTrump',
 'TheEconomist',
 'aplusk',
 'HillaryClinton',
 'TechCrunch',
 'elonmusk',
 'NewYorker',
 'mcuban',
 'jack',
 'MarketWatch',
 'CNBC',
 'ForbesTech',
 'sacca',
 'SAI',
 'paulg',
 'themotleyfool',
 'ReformedBroker',
 'StockTwits',
 'cnntech',
 'MONEY',
 'Carl Icahn',
 'sgblank',
 'investorslive',
 'markflowchatter',
 'MarkYusko',
 'FinancialTimes',
 'tim_cook']

**Problem:** Most codes on the internet use the Access Token Auth method, which is limited to 180 Requests/15 mins limit (18,000 tweets/15 mins). If you download 18K tweets before 15 mins, you won’t be able to get any more results until your 15 min window expires and you search again.

**Solution:** Use Application only Auth instead of the Access Token Auth. Application only auth has higher limits - 450 request/sec (45,000 tweets/15-min), which is 2.5 times more than the Access Token Limit.

The secret is the AppAuthHandler instead of the more frequent OAuthHandler which you find being used in lots of code samples. This sets up App-only Auth and gives you higher limits. Also as an added bonus notice the wait_on_rate_limit & wait_on_rate_limit_notify flags set to true. What this does is make the Tweepy API call auto wait (sleep) when it hits the rate limit and continue upon expiry of the window. This avoids you to have to program this part manually, which as you’ll shortly see makes your program much more simple and elegant.

In [14]:
handle = ['aplusk', 'BarackObama']

In [15]:
handle

['aplusk', 'BarackObama']

In [16]:
import sys

# Replace the API_KEY and API_SECRET with your application's key and secret:
auth = tweepy.AppAuthHandler(my_keys['CONSUMER_KEY'], my_keys['CONSUMER_SECRET'])

# Authorize twitter, initialize tweepy
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [20]:
import csv

maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 200  # this is the max the API permits
fName = 'tweets.csv' # We'll store the tweets in a csv file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1E10

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry)
                else:
                    new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
                                            since_id=sinceId)
            else:

                if (not sinceId):
                    new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.user_timeline(screen_name = 'aplusk', count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                outtweets = [[tweet.user.screen_name, tweet.created_at, tweet.text.encode("utf-8")]]
                writer = csv.writer(f)
                writer.writerow(["id","created_at","text"])
                writer.writerows(outtweets)
              
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id  

        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("\n Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

Downloaded 200 tweets
Downloaded 400 tweets
Downloaded 600 tweets
Downloaded 799 tweets
Downloaded 999 tweets
Downloaded 1199 tweets
Downloaded 1399 tweets
Downloaded 1599 tweets
Downloaded 1799 tweets
Downloaded 1999 tweets
Downloaded 2198 tweets
Downloaded 2398 tweets
Downloaded 2598 tweets
Downloaded 2798 tweets
Downloaded 2998 tweets
Downloaded 3198 tweets
Downloaded 3243 tweets
No more tweets found

 Downloaded 3243 tweets, Saved to tweets.csv


In [17]:
import csv

maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 200  # this is the max the API permits
fName = 'tweets.csv' # We'll store the tweets in a csv file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1E10

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            for h in handle:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = h, count=tweetsPerQry)
                    else:
                        new_tweets = api.user_timeline(screen_name = h, count=tweetsPerQry,
                                            since_id=sinceId)
                else:

                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = h, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                    else:
                        new_tweets = api.user_timeline(screen_name = h, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                for tweet in new_tweets:
                    outtweets = [[tweet.user.screen_name, tweet.created_at, tweet.text.encode("utf-8")]]
                    writer = csv.writer(f)
                    writer.writerow(["id","created_at","text"])
                    writer.writerows(outtweets)
              
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id  

        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("\n Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

Downloaded 200 tweets
Downloaded 400 tweets
Downloaded 600 tweets
Downloaded 800 tweets
Downloaded 1000 tweets
Downloaded 1200 tweets
Downloaded 1400 tweets
Downloaded 1600 tweets
Downloaded 1800 tweets
Downloaded 2000 tweets
Downloaded 2200 tweets
Downloaded 2399 tweets
Downloaded 2599 tweets
Downloaded 2796 tweets
Downloaded 2996 tweets
Downloaded 3196 tweets
Downloaded 3233 tweets
No more tweets found
Downloaded 3233 tweets


IndexError: list index out of range

In [ ]:
all_tweets = pd.read_csv('tweets.csv')
all_tweets['id'].unique()

In [ ]:
!rm tweets.csv